In [39]:
using Revise
using Catan
using CatanLearning
using Random
using CSV
using MLJ
using DataFrames
import DataFramesMeta as DFM
using DelimitedFiles
import MLJModelInterface
const MMI = MLJModelInterface

MLJModelInterface

In [40]:
function naive_model_proba(X_in)
    X = X_in[1]
    #println("analyzing $X")
    unscaled_feature = X[!,:CountSettlement] + X[!,:CountCity] + 2*(X[!,:HasLargestArmy] + X[!,:HasLongestRoad])
    return min.(unscaled_feature ./ 10, 1.0)
end

MMI.@mlj_model mutable struct NaivePublicVictoryPoints <: MLJModelInterface.Deterministic
end

MMI.fit(::NaivePublicVictoryPoints, verbosity, data...) = (nothing, nothing, nothing)
MMI.predict(model::NaivePublicVictoryPoints, fitresult, new_data...) = naive_model_proba(new_data)

model = NaivePublicVictoryPoints()
naive_machine = machine(model, nothing, nothing) |> MLJ.fit!

trained Machine; caches model-specific representations of data
  model: NaivePublicVictoryPoints()
  args: 
    1:	Source @978 ⏎ Nothing
    2:	Source @471 ⏎ Nothing


In [52]:
config_file = "configs/feature_generation.toml"
(configs, player_configs, logger) = Catan.parse_configs(config_file)
rf_machine = CatanLearning.try_load_public_model_from_csv(:any, configs)
#rf_machine = CatanLearning.train_model_from_csv(CatanLearning.load_tree_model(), player_configs["PUBLIC_FEATURES"], num_tuning_iterations = 1)

Savefile set to savefile.txt
Configs loaded from configs/feature_generation.toml


trained Machine; caches model-specific representations of data
  model: RandomForestClassifier(max_depth = -1, …)
  args: 


In [46]:
test_data_csv = "./data/public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)
y, X = unpack(df, ==(:WonGame));
typeof(X)
for col in names(X)
    println(col)
end
length(names(df))

CountSettlement
CountTotalSettlement
CountCity
CountRoad
SumWoodDiceWeight
SumBrickDiceWeight
SumPastureDiceWeight
SumStoneDiceWeight
SumGrainDiceWeight
CountPortWood
CountPortBrick
CountPortPasture
CountPortStone
CountPortGrain
CountTotalWood
CountTotalBrick
CountTotalPasture
CountTotalStone
CountTotalGrain
CountDevCardsKnight
CountDevCardsMonopoly
CountDevCardsYearOfPlenty
CountDevCardsRoadBuilding
CountDevCardsVictoryPoint
HasLargestArmy
HasLongestRoad


27

In [49]:
test_data_csv = "./data/public_features.csv"
df = CatanLearning.load_typed_features_from_csv(test_data_csv)

Row,CountSettlement,CountTotalSettlement,CountCity,CountRoad,SumWoodDiceWeight,SumBrickDiceWeight,SumPastureDiceWeight,SumStoneDiceWeight,SumGrainDiceWeight,CountPortWood,CountPortBrick,CountPortPasture,CountPortStone,CountPortGrain,CountDevCardsUsedKnight,CountDevCardsUsedMonopoly,CountDevCardsUsedYearOfPlenty,CountDevCardsUsedRoadBuilding,HasLargestArmy,HasLongestRoad,CountVisibleVictoryPoint,HasMostPoints,WonGame
,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Int8,Bool,Bool,Int8,Bool,Cat…
1,2,2,0,4,0,2,0,5,0,0,0,1,0,0,1,0,0,0,false,false,2,false,false
2,2,2,0,14,4,2,2,0,0,0,0,0,0,0,0,0,0,0,false,true,4,false,false
3,2,2,0,2,3,0,0,4,0,0,0,0,0,0,1,0,0,0,false,false,2,false,false
4,0,2,2,5,0,0,16,10,8,0,0,0,0,0,8,2,2,1,true,false,6,true,true
5,1,4,3,14,6,6,18,34,8,0,0,0,0,0,3,1,1,1,false,true,9,true,true
6,2,2,0,8,4,0,2,0,7,0,0,0,0,0,3,0,1,1,true,false,4,false,false
7,3,3,0,11,9,1,0,0,0,1,0,0,0,0,1,0,0,0,false,false,3,false,false
8,2,2,0,8,0,3,0,0,2,0,0,0,0,0,1,1,0,0,false,false,2,false,false
9,1,3,2,8,1,0,26,10,23,0,0,0,0,0,7,1,1,2,true,false,7,true,true


In [6]:

rf_res = MLJ.predict(rf_machine, X)


4120-element CategoricalDistributions.UnivariateFiniteVector{OrderedFactor{2}, Bool, UInt32, Float64}:
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.769, true=>0.231)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.846, true=>0.154)
 UnivariateFinite{OrderedFactor{2}}(false=>0.462, true=>0.538)
 UnivariateFinite{OrderedFactor{2}}(false=>0.154, true=>0.846)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.615, true=>0.385)
 ⋮
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)
 UnivariateFinite{OrderedFactor{2}}(false=>0.0769, true=>0.923)
 UnivariateFinite{OrderedFactor{2}}(false=>

In [ ]:
import Catan: PlayerType, DefaultRobotPlayer
players = Vector{PlayerType}([
    CatanLearning.EmpathRobotPlayer(:Blue, configs), 
    DefaultRobotPlayer(:Test2, configs),
    DefaultRobotPlayer(:Test3, configs),
    DefaultRobotPlayer(:Test4, configs),
   ])
player = players[1]
board = Board(configs)


Board(Dict(:N => 6, :F => 9, :M => 5, :A => 12, :D => 8, :R => 11, :B => 2, :Q => 9, :P => 5, :I => 10…), Dict(5 => [:M, :P], 7 => [:S], 12 => [:A], 8 => [:D, :K], 4 => [:E, :O], 6 => [:N, :L], 2 => [:B], 11 => [:R, :J], 10 => [:I, :H], 9 => [:F, :Q]…), Dict(:N => :Grain, :F => :Wood, :M => :Grain, :A => :Brick, :D => :Pasture, :R => :Brick, :B => :Stone, :Q => :Stone, :P => :Wood, :I => :Pasture…), Dict{Tuple, Catan.Building}(), Dict{Tuple, Set{Road}}(), Dict{Tuple, Symbol}((1, 2) => :Stone, (6, 5) => :All, (2, 8) => :All, (4, 2) => :Brick, (6, 2) => :All, (1, 4) => :Grain, (5, 9) => :Pasture, (6, 4) => :All, (5, 8) => :Pasture, (3, 2) => :All…), Any[[nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing], [nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 

In [51]:
p_public = PlayerPublicView(player)
f = CatanLearning.compute_public_features(board, p_public)
#CatanLearning.predict_public_model(rf_machine, board, p_public)


21-element Vector{Pair{Symbol, Float64}}:
               :CountSettlement => 0.0
          :CountTotalSettlement => 0.0
                     :CountCity => 0.0
                     :CountRoad => 0.0
             :SumWoodDiceWeight => 0.0
            :SumBrickDiceWeight => 0.0
          :SumPastureDiceWeight => 0.0
            :SumStoneDiceWeight => 0.0
            :SumGrainDiceWeight => 0.0
                 :CountPortWood => 0.0
                                ⋮
                :CountPortStone => 0.0
                :CountPortGrain => 0.0
       :CountDevCardsUsedKnight => 0.0
     :CountDevCardsUsedMonopoly => 0.0
 :CountDevCardsUsedYearOfPlenty => 0.0
 :CountDevCardsUsedRoadBuilding => 0.0
                :HasLargestArmy => 0.0
                :HasLongestRoad => 0.0
      :CountVisibleVictoryPoint => 0.0

In [58]:
X_new = DataFrame(f)
CatanLearning.coerce_feature_types!(X_new)
p = predict(rf_machine, X_new)

1-element CategoricalDistributions.UnivariateFiniteVector{OrderedFactor{2}, Bool, UInt32, Float64}:
 UnivariateFinite{OrderedFactor{2}}(false=>1.0, true=>0.0)

In [67]:
pdf(p[1], 3)

DomainError: DomainError with Value 3 not in pool. :


In [ ]:
#MLJ.predict(naive_machine, X)
#MLJ.save(naive_machine, "../CatanLearning/data/naive_public_model.jls")
predict(rf_machine, f)

In [10]:
mach = naive_machine
typeof(mach)

Machine{NaivePublicVictoryPoints, NaivePublicVictoryPoints, true}

In [11]:
typeof(mach.args[1])

MLJBase.Source